## Import Libraries

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import os
import time
from Utils import homepath


import warnings
warnings.filterwarnings('ignore')

## Fetch Data

### Index

In [ ]:
# Define the S&P 500 and S&P 600 index symbols
indice_list = ["^GSPC", "SPSM"]

# Download historical data
indices = yf.download(indice_list, start="2020-01-01", end="2025-01-01")["Close"]


indices.columns = ["sp600","sp500"]
# indices.to_csv(homepath + 'data/indices.csv')

[*********************100%***********************]  2 of 2 completed

1 Failed download:
['SPSM']: SSLError(MaxRetryError("HTTPSConnectionPool(host='www.yahoo.com', port=443): Max retries exceeded with url: /?guccounter=1 (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1000)')))"))


### risk free rate

In [ ]:
# Define risk-free rate tickers
risk_free_tickers = {
    "3M T-Bill Rate": "^IRX",   # 3-month risk-free rate
    "10Y Treasury Yield": "^TNX"  # 10-year bond yield
}

test = ["^IRX","^TNX"]

# Download historical data (latest 1 year)
rf_data = yf.download(list(risk_free_tickers.values()), start="2020-01-01", end="2025-01-01")["Close"]

# Rename columns for clarity
rf_data.columns = risk_free_tickers.keys()

# Convert percentages to decimal form (Yahoo Finance reports these as percentages)
rf_data = rf_data / 100 

# rf_data.to_csv(homepath + "data/interest_rates.csv")


[*********************100%***********************]  2 of 2 completed


### 30 Random Tickers timeseries

In [ ]:
def get_stock_data(tickers, start_date, end_date):
    """
    Fetch adjusted close (or close) prices for a list of stock tickers from Yahoo Finance.
    
    Parameters:
    tickers (list): List of stock tickers (e.g., ['AAPL', 'MSFT']).
    start_date (str): Start date in 'YYYY-MM-DD' format.
    end_date (str): End date in 'YYYY-MM-DD' format.
    
    Returns:
    DataFrame: A pandas DataFrame with prices for the selected tickers.
    """
    raw_data = yf.download(tickers, start=start_date, end=end_date, progress=False)

    if 'Adj Close' in raw_data:
        stock_data = raw_data['Adj Close']
    elif 'Close' in raw_data:
        stock_data = raw_data['Close']
    else:
        raise ValueError("Neither 'Adj Close' nor 'Close' data is available for the tickers.")
    
    stock_data.dropna(axis=1, inplace=True)
    return stock_data

In [16]:
all_tickers = [
    'AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META', 'TSLA', 'NVDA', 'JPM', 'V', 'JNJ',
    'WMT', 'PG', 'DIS', 'BAC', 'XOM', 'CVX', 'PFE', 'KO', 'PEP', 'NFLX',
    'ADBE', 'CSCO', 'INTC', 'CRM', 'ORCL', 'QCOM', 'IBM', 'AVGO', 'TXN', 'T'
]

start_date = "2020-01-01"
end_date = "2023-01-01" 

try:
    stock_prices = get_stock_data(all_tickers, start_date, end_date)
    print(f"Successfully fetched data for {len(stock_prices.columns)} tickers.")
except Exception as e:
    print(f"An error occurred: {e}")

Successfully fetched data for 30 tickers.


In [5]:
# stock_prices.to_csv(homepath + "data/stock_prices_30_tickers.csv")

### SP500 Tickers Timeseries

In [6]:
mock_tickers = pd.read_excel(homepath + "data/SP500Tickers.xlsx")["Symbol"].to_list()

In [37]:
# Batch size
batch_size = 30

def save_data_to_file(data, filename):
    data.to_csv(filename, sep='\t', index=True)

# Loop through batches of the tickers
for i in range(0, len(mock_tickers), batch_size):
    batch = mock_tickers[i:i + batch_size]  # Get the next 30 tickers
    print(f"Downloading batch {i // batch_size + 1}: {batch}")
    
    # Download data for the current batch
    try:
        data = yf.download(batch, start="2020-01-01", end="2025-01-01")  # Adjust date range
        if 'Adj Close' in data:
            stock_data = data['Adj Close']
        elif 'Close' in data:
            stock_data = data['Close']
        else:
            raise ValueError("Neither 'Adj Close' nor 'Close' data is available for the tickers.")
        # Save the data to a text file
        batch_filename = homepath + f"data/batch_{i // batch_size + 1}.txt"
        save_data_to_file(stock_data, batch_filename)
        print(f"Batch {i // batch_size + 1} data saved to {batch_filename}")
        
    except Exception as e:
        print(f"Error downloading/saving data for batch {i // batch_size + 1}: {e}")

print("All batches processed.")

[*********************100%***********************]  30 of 30 completed


Batch 1 data saved to C:/A.PROJECTS/stockprediction/data/batch_1.txt


[*********************100%***********************]  30 of 30 completed


Batch 2 data saved to C:/A.PROJECTS/stockprediction/data/batch_2.txt


[*********************100%***********************]  30 of 30 completed

2 Failed downloads:
['BF.B']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2020-01-01 -> 2025-01-01)')
['BRK.B']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')


...

Batch 15 data saved to C:/A.PROJECTS/stockprediction/data/batch_15.txt


[*********************100%***********************]  30 of 30 completed


Batch 16 data saved to C:/A.PROJECTS/stockprediction/data/batch_16.txt


[*********************100%***********************]  23 of 23 completed


Batch 17 data saved to C:/A.PROJECTS/stockprediction/data/batch_17.txt
All batches processed.


In [9]:
def combine_batch_files(num_batches):
    # List to hold DataFrames
    all_dataframes = []
    
    # Iterate over each batch file
    for batch_number in range(1, num_batches + 1):
        batch_filename = homepath + f"data/batch_{batch_number}.txt"
        
        # Check if the file exists
        if os.path.exists(batch_filename):
            print(f"Loading {batch_filename}")
            
            # Read the batch file into a DataFrame
            data = pd.read_csv(batch_filename, sep='\t', index_col=0, parse_dates=True)
            # Append DataFrame to the list
            all_dataframes.append(data)
        else:
            print(f"{batch_filename} does not exist.")
    
    # Concatenate all DataFrames in the list into a single DataFrame
    combined_data = pd.concat(all_dataframes, axis=1)
    
    return combined_data


number_of_batches = 17  # For example, replace with actual number of batch files you created

# Combine all batch data files into one DataFrame
combined_df = combine_batch_files(number_of_batches)

# Optional: Save the combined DataFrame to a file
combined_df_filename = homepath + "data/sp500_timeseries.txt"
print(f"Combined data saved to {combined_df_filename}")
# Example output of the combined DataFrame
print(combined_df.shape)

combined_df.to_csv(combined_df_filename, sep='\t')

Loading C:/A.PROJECTS/stockprediction/data/batch_1.txt
Loading C:/A.PROJECTS/stockprediction/data/batch_2.txt
Loading C:/A.PROJECTS/stockprediction/data/batch_3.txt
Loading C:/A.PROJECTS/stockprediction/data/batch_4.txt
Loading C:/A.PROJECTS/stockprediction/data/batch_5.txt
Loading C:/A.PROJECTS/stockprediction/data/batch_6.txt
Loading C:/A.PROJECTS/stockprediction/data/batch_7.txt
Loading C:/A.PROJECTS/stockprediction/data/batch_8.txt
Loading C:/A.PROJECTS/stockprediction/data/batch_9.txt
Loading C:/A.PROJECTS/stockprediction/data/batch_10.txt
Loading C:/A.PROJECTS/stockprediction/data/batch_11.txt
Loading C:/A.PROJECTS/stockprediction/data/batch_12.txt
Loading C:/A.PROJECTS/stockprediction/data/batch_13.txt
Loading C:/A.PROJECTS/stockprediction/data/batch_14.txt
Loading C:/A.PROJECTS/stockprediction/data/batch_15.txt
Loading C:/A.PROJECTS/stockprediction/data/batch_16.txt
Loading C:/A.PROJECTS/stockprediction/data/batch_17.txt
Combined data saved to C:/A.PROJECTS/stockprediction/data

### SP600 Tickers Timeseries

In [4]:
sp600_tickers = pd.read_csv(homepath + "data/SP600Tickers.csv")["Symbol"].to_list()

In [5]:
# Batch size
batch_size = 30

def save_data_to_file(data, filename):
    data.to_csv(filename, sep='\t', index=True)


sp600_list = []
# Loop through batches of the tickers
for i in range(0, len(sp600_tickers), batch_size):
    batch = sp600_tickers[i:i + batch_size]  # Get the next 30 tickers
    print(f"Downloading batch {i // batch_size + 1}: {batch}")
    
    # Download data for the current batch
    try:
        data = yf.download(batch, start="2020-01-01", end="2025-01-01")  # Adjust date range
        if 'Adj Close' in data:
            stock_data = data['Adj Close']
        elif 'Close' in data:
            stock_data = data['Close']
        else:
            raise ValueError("Neither 'Adj Close' nor 'Close' data is available for the tickers.")
        # Save the data to a text file
        batch_filename = homepath + f"data/batch_{i // batch_size + 1}.txt"
        # save_data_to_file(stock_data, batch_filename)
        sp600_list.append(stock_data)
        print(f"Batch {i // batch_size + 1} data saved to {batch_filename}")
        
    except Exception as e:
        print(f"Error downloading/saving data for batch {i // batch_size + 1}: {e}")

print("All batches processed.")

[*********************100%***********************]  28 of 30 completed

2 Failed downloads:
['ABR', 'ACIW']: SSLError(MaxRetryError("HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Max retries exceeded with url: /v8/finance/chart/%ticker%?period1=1577854800&period2=1735707600&interval=1d&includePrePost=False&events=div%2Csplits%2CcapitalGains&crumb=AzA6uKTvfzG (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1000)')))"))
[***                    7%                       ]  2 of 30 completed

Batch 1 data saved to C:/A.PROJECTS/stockprediction/data/batch_1.txt


[*********************100%***********************]  30 of 30 completedd


Batch 2 data saved to C:/A.PROJECTS/stockprediction/data/batch_2.txt


[*********************100%***********************]  30 of 30 completed


...

Batch 19 data saved to C:/A.PROJECTS/stockprediction/data/batch_19.txt


[*********************100%***********************]  30 of 30 completed


Batch 20 data saved to C:/A.PROJECTS/stockprediction/data/batch_20.txt


[*********************100%***********************]  2 of 2 completed

Batch 21 data saved to C:/A.PROJECTS/stockprediction/data/batch_21.txt
All batches processed.


In [ ]:
sp600_timeseries = pd.concat(sp600_list, axis=1)
sp600_timeseries = sp600_timeseries.dropna(axis=1, thresh=len(sp600_timeseries)-20)
# sp600_timeseries.to_csv(homepath+'data/sp600_timeseries.csv')

### NYSE All Tickers Timeseries